In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
#### Merging 12 months of sales data into a single CSV file

In [ ]:
os.listdir(os.getcwd())

In [ ]:
os.getcwd()

In [ ]:
#list comprehension to locate files in folder 
files = [file for file in os.listdir("/kaggle/input/sales-analysis")] 

cmonthdata = pd.DataFrame()

for file in files:
    db = pd.read_csv("/kaggle/input/sales-analysis/"+file)
    cmonthdata= pd.concat((cmonthdata,db))
    
cmonthdata.to_csv("allmdata.csv",index=False)

#### Read updated data

In [ ]:
alldata = pd.read_csv("allmdata.csv")
alldata.head()

#### Clean Data

In [ ]:
missdat = alldata[alldata.isna().any(axis=1)] #data base with missing values of original data
alldata = alldata.dropna(how='all')
alldata.head()

#### Deal with the 'Or' found in Month column

In [ ]:
alldata = alldata[alldata["Order Date"].str[0:2] != "Or"]

#### Add month column 

In [ ]:
alldata['Month'] = alldata["Order Date"].str[0:2] # extract the month number from date
alldata.loc[:,'Month'] = alldata['Month'].astype('int32') # convert to int
alldata["Quantity Ordered"] = pd.to_numeric(alldata['Quantity Ordered']) #Otra forma de hacerlo
alldata["Price Each"] = alldata['Price Each'].astype('float32') # decimals
alldata.dtypes

In [ ]:
# year column
alldata['Year'] = pd.to_numeric(alldata["Order Date"].str[6:8])
alldata.head()

#### What was the best month for sales? How much was earned that month?

In [ ]:
# add sales column
alldata['Sales'] = alldata["Quantity Ordered"] * alldata['Price Each']
alldata.head()

In [ ]:
alldata.groupby(['Year','Month']).sum()

In [ ]:
# graph form 1
plt.figure(figsize=(10,7))
result = alldata.groupby(['Year','Month']).sum()
result['Sales'].plot.bar()
;

In [ ]:
# graph form 2
plt.figure(figsize=(10,7))
sns.set_style("whitegrid") #Para gridlines
bplot = sns.barplot('Month','Sales',hue="Year",estimator = np.sum,color='r',data=alldata)

plt.ticklabel_format(style='plain', axis='y') #drop scientific notation from y axis

# para cambiar los valores del eje y a valores con coma
import matplotlib as mpl
bplot.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))

#
sns.set_context("notebook", font_scale=2) # tamaño letras
;

#### What city had the highest number of sales?

In [ ]:
alldata.head()

In [ ]:
alldata["City"] = [i.split(",")[1] + "," + i.split(",")[-1].split(" ")[1]
                   for i in alldata['Purchase Address'].tolist()] #Extraer ciudad de direccion

# Otra Forma #
def getcity(address):
    return address.split(",")[1]

def getstate(address):   # se añade estado para diferenciar dos ciudades que se llaman igual(Portland)
    return address.split(",")[-1].split(" ")[1] # hay un espacio antes de las siglas de los Estados

alldata["city"] = alldata['Purchase Address'].apply(lambda x: f"{getcity(x)}({getstate(x)})")
alldata.head()

In [ ]:
alldata.groupby("city").sum()["Sales"]

In [ ]:
plt.figure(figsize=(15,7))
citybplot = sns.barplot("City","Sales",estimator = np.sum,color='b',data=alldata)
sns.set_context("notebook", font_scale=1.5)
plt.ticklabel_format(style='plain', axis='y')
citybplot.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.xticks(rotation=45) # rotar valores eje
;

#### What time should we display ads to max likelihood of customer's buying product?

In [ ]:
alldata['Order Date'] = pd.to_datetime(alldata['Order Date'])

In [ ]:
alldata["hour"]=alldata['Order Date'].dt.hour # create hour column
alldata.head()

In [ ]:
plt.figure(figsize=(15,7))
## Otra forma de hacer countplot con linechart              #  este \ es para seguir la formula mas abajo

dhour = alldata.groupby(['hour'],as_index=False).count()

hourlplot = sns.lineplot("hour","Sales",color='g',data=dhour) \
.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.xticks(dhour.index)
;

In [ ]:
alldata.groupby(['hour'],as_index=False).count().index

In [ ]:
plt.figure(figsize=(15,7))
## Forma countplot
sns.countplot("hour",color='g',data=alldata) \
.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
;

####  What products are most often sold together ?

In [ ]:
sameprod = alldata[alldata['Order ID'].duplicated(keep=False)] #create dataframe with duplicates

#create new column with combo prod in Order IDs
sameprod['combo'] = sameprod.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))

sameprod = sameprod[['Order ID','combo']].drop_duplicates() # remove duplicates 
sameprod.head()

In [ ]:
#Count unique pairs of numbers
#ref : https://stackoverflow.com/questions/52195887/counting-unique-pairs-of-numbers-into-a-python-dictionary
from itertools import combinations
from collections import Counter

count = Counter()

for row in sameprod['combo']:
    rowl= row.split(',')
    count.update(Counter(combinations(rowl,2))) # most common pair in column
    
count.most_common(5)

#### What product sold the most and why?

In [ ]:
plt.figure(figsize=(16,7))
alldata.groupby('Product')['Quantity Ordered'].sum().plot.bar() \
.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.xticks(rotation=80,size=14)

#multieje
plt.axes().twinx()
alldata.groupby('Product')['Price Each'].mean().plot(color='r') \
.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
;